In [1]:
import numpy as np
import pandas as pd
import time
from collections import OrderedDict
from sklearn import preprocessing
from matplotlib import pyplot as plt
import annoy

# Data Preparation

In [ ]:
%%time
#200000 compounds with 469 features
all_features = pd.read_csv("c2vpoint2m.txt.gz", sep="\t", header=None)

In [ ]:
#make the compound names the index
all_features.set_index(0, inplace=True)

In [ ]:
%%time
#725634 compounds with target and activity
cgan = pd.read_csv("activities.txt.gz", sep="\t")

In [ ]:
#we are only interested in the compounds whose target is EGFR
egfr_activity = cgan[cgan['target'] == 'EGFR'].set_index('compound')

In [ ]:
#separate into two dataframes, one with active compounds
egfr_active = egfr_activity[egfr_activity['activity'] >= 6]
egfr_active_compounds = egfr_active.index.values

In [ ]:
#and one with inactive ones
egfr_inactive = egfr_activity[egfr_activity['activity']  < 6]
egfr_inactive_compounds = egfr_inactive.index.values

In [ ]:
#5275 compounds with EGFR target and 469 features
egfr_features = pd.read_csv("egfr.c2v.txt", sep="\t", header=None).set_index(0)

In [ ]:
#isolate the names for easier access
egfr_compounds = egfr_features.index.values

In [ ]:
#make dataframes with features for the active and inactive compounds
egfr_active_features = egfr_features.loc[egfr_active_compounds]
egfr_inactive_features = egfr_features.loc[egfr_inactive_compounds]

In [ ]:
#scale the data so that each feature is normalized and can be compared to other features
min_max_scaler = preprocessing.MinMaxScaler() #fits data between 0 and 1

In [ ]:
#normalize the active compounds
egfr_active_features_norm = pd.DataFrame(min_max_scaler.fit_transform(egfr_active_features), 
                                         index=egfr_active_features.index)

In [ ]:
#normalize the inactive compounds
egfr_inactive_features_norm = pd.DataFrame(min_max_scaler.fit_transform(egfr_inactive_features), 
                                           index=egfr_inactive_features.index)

In [ ]:
#remove duplicates from all_features - can also use DataFrame.drop_duplicates()
all_features_cleaned = all_features.loc[list(set(all_features.index.values) - set(egfr_compounds))]

In [ ]:
#normalize the database
all_features_cleaned_norm = pd.DataFrame(min_max_scaler.fit_transform(all_features_cleaned), index=all_features_cleaned.index)

# Logic Creation

In [ ]:
def dataprep(active, rest, size, seed):
    #create the query from a random sample of the data
    size = int(active.shape[0] * size + .5)
    query = active.sample(n=size, random_state=seed)
    
    #remove the query, then concatenate the remaining data for training
    sub = list(set(active.index.values) - set(query.index.values))
    active_minus_query = active.loc[sub]
    database = pd.concat([active_minus_query] + rest)
    
    return database, query

In [ ]:
def buildAnnoyModel(database, n_neighbors, metric, n_trees):
    #create model - first argument is the length of the vectors
    index = annoy.AnnoyIndex(database.shape[1], metric=metric)
    
    #add training data
    start = time.time()
    
    #note that this is much slower than the other models because the data must be
    #added row-by-row instead of the entire database at once
    for i in range(database.shape[0]):
        index.add_item(i, database.loc[database.index.values[i]])
    
    #build trees
    index.build(n_trees)
    end = time.time()
    
    build_time = end - start
    
    return index, build_time

In [ ]:
def runAnnoyQuery(model, query, n_neighbors, database, active):
    indices = []
    
    #run query
    start = time.time()
    for j in range(query.shape[0]):
        i = model.get_nns_by_vector(query.loc[query.index.values[j]], n_neighbors)
        indices.append(i)
    end = time.time()
    
    query_time  = end - start
    
    #assess the quality of the results - how many of the neighbors are in the success dataframe
    sum = 0
    for i in indices:
        count = 0
        for j in i:
            if database.index.values[j] in active.index.values:
                count += 1
        sum += count / float(k)
    average = sum / query.shape[0]
    
    return average, query_time

In [ ]:
def annoyNN(active, rest, size, n_neighbors, metric, n_trees, seed):
    #prepare the data
    database, query = dataprep(active, rest, size, seed)
    
    #build the model
    model, build_time = buildAnnoyModel(database, n_neighbors, metric, n_trees)
    
    #run the query
    average, query_time = runAnnoyQuery(model, query, n_neighbors, database, active)
 
    #return the results in a table
    d = OrderedDict({'algorithm/index':['Annoy' + ' (' + str(n_trees) + ' trees)'], 'metric':[metric], 
                     'n_neighbors':[n_neighbors], 'query size':[size], 'build time (s)':[build_time], 
                     'query time (s)':[query_time], 'quality':[average], 'seed':[seed]})
    
    return pd.DataFrame(data=d)

# Tests

In [2]:
annoy_results = pd.DataFrame()

In [ ]:
a = annoyNN(egfr_active_features_norm, [stuff_cleaned_norm, egfr_inactive_features_norm], 0.2, 100, 'euclidean', 10, 0)

In [ ]:
temp = []
for metric in ['angular', 'euclidean', 'manhattan', 'hamming']:
    for n_trees in [5, 10, 20]:
        temp.append(annoyNN(active=egfr_active_features_norm, rest=[stuff_cleaned_norm, egfr_inactive_features_norm], 
                            size=0.2, n_neighbors=100, metric=metric, n_trees=n_trees, seed=0))
a = pd.concat(temp)
a

In [ ]:
annoy_results

# Graphs


In [ ]:
plt.figure()
for i in range(12):
    plt.scatter(a[i:i+1]['query time (s)'], a[i:i+1]['quality'], s=10)
plt.ylim(0.995, 1)
plt.xlabel('query time (s)')
plt.ylabel('quality')
plt.legend(['annoy/angular/5', 'annoy/angular/10', 'annoy/angular/15', 
            'annoy/euclidean/5', 'annoy/euclidean/10', 'annoy/euclidean/15', 
            'annoy/manhattan/5', 'annoy/manhattan/10', 'annoy/manhattan/15', 
            'annoy/hamming/5', 'annoy/hamming/10', 'annoy/hamming/15'])
plt.show()

In [ ]:
plt.figure()
for i in range(12):
    plt.scatter(a[i:i+1]['build time (s)'], a[i:i+1]['quality'], s=10)
plt.ylim(0.995, 1)
plt.xlabel('build time (s)')
plt.ylabel('quality')
plt.legend(['annoy/angular/5', 'annoy/angular/10', 'annoy/angular/15', 
            'annoy/euclidean/5', 'annoy/euclidean/10', 'annoy/euclidean/15', 
            'annoy/manhattan/5', 'annoy/manhattan/10', 'annoy/manhattan/15', 
            'annoy/hamming/5', 'annoy/hamming/10', 'annoy/hamming/15'])
plt.show()